# Fraud Detection with Amazon SageMaker FeatureStore


---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-2/sagemaker-featurestore|sagemaker_featurestore_fraud_detection_python_sdk.ipynb)

---


Kernel `Python 3 (Data Science)` works well with this notebook.

The following policies need to be attached to the execution role:
- AmazonSageMakerFullAccess
- AmazonS3FullAccess

## Contents
1. [Background](#Background)
1. [Setup SageMaker FeatureStore](#Setup-SageMaker-FeatureStore)
1. [Inspect Dataset](#Inspect-Dataset)
1. [Ingest Data into FeatureStore](#Ingest-Data-into-FeatureStore)
1. [Build Training Dataset](#Build-Training-Dataset)
1. [Train and Deploy the Model](#Train-and-Deploy-the-Model)
1. [SageMaker FeatureStore At Inference](#SageMaker-FeatureStore-During-Inference)
1. [Cleanup Resources](#Cleanup-Resources)

## Background

Amazon SageMaker FeatureStore is a new SageMaker capability that makes it easy for customers to create and manage curated data for machine learning (ML) development. SageMaker FeatureStore enables data ingestion via a high TPS API and data consumption via the online and offline stores. 

This notebook provides an example for the APIs provided by SageMaker FeatureStore by walking through the process of training a fraud detection model. The notebook demonstrates how the dataset's tables can be ingested into the FeatureStore, queried to create a training dataset, and quickly accessed during inference. 


### Terminology

A **FeatureGroup** is the main resource that contains the metadata for all the data stored in SageMaker FeatureStore. A FeatureGroup contains a list of FeatureDefinitions. A **FeatureDefinition** consists of a name and one of the following data types: a integral, string or decimal. The FeatureGroup also contains an **OnlineStoreConfig** and an **OfflineStoreConfig** controlling where the data is stored. Enabling the online store allows quick access to the latest value for a Record via the GetRecord API. The offline store, a required configuration, allows storage of historical data in your S3 bucket. 

Once a FeatureGroup is created, data can be added as Records. **Records** can be thought of as a row in a table. Each record will have a unique **RecordIdentifier** along with values for all other FeatureDefinitions in the FeatureGroup. 

## Setup SageMaker FeatureStore

Let's start by setting up the SageMaker Python SDK and boto client. Note that this notebook requires a `boto3` version above `1.17.21`

In [1]:
import boto3
import sagemaker

# original_boto3_version = boto3.__version__
# %pip install 'boto3>1.17.21'

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
from sagemaker.session import Session

region = boto3.Session().region_name

boto_session = boto3.Session(region_name=region)

sagemaker_client = boto_session.client(service_name="sagemaker", region_name=region)
featurestore_runtime = boto_session.client(
    service_name="sagemaker-featurestore-runtime", region_name=region
)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime,
)

#### S3 Bucket Setup For The OfflineStore

SageMaker FeatureStore writes the data in the OfflineStore of a FeatureGroup to a S3 bucket owned by you. To be able to write to your S3 bucket, SageMaker FeatureStore assumes an IAM role which has access to it. The role is also owned by you.
Note that the same bucket can be re-used across FeatureGroups. Data in the bucket is partitioned by FeatureGroup.

Set the default s3 bucket name and it will be referenced throughout the notebook.

In [3]:
# You can modify the following to use a bucket of your choosing
default_s3_bucket_name = feature_store_session.default_bucket()
prefix = "sagemaker-featurestore-demo"

print(default_s3_bucket_name)

sagemaker-us-east-1-087941614028


Set up the IAM role. This role gives SageMaker FeatureStore access to your S3 bucket. 

<div class="alert alert-block alert-warning">
<b>Note:</b> In this example we use the default SageMaker role, assuming it has both <b>AmazonSageMakerFullAccess</b> and <b>AmazonSageMakerFeatureStoreAccess</b> managed policies. If not, please make sure to attach them to the role before proceeding.
</div>

In [4]:
from sagemaker import get_execution_role

# You can modify the following to use a role of your choosing. See the documentation for how to create this.
role = get_execution_role()
print(role)

arn:aws:iam::087941614028:role/LabRole


## Inspect Dataset

The provided dataset is a synthetic dataset with two tables: identity and transactions. They can both be joined by the `TransactionId` column. The transaction table contains information about a particular transaction such as amount, credit or debit card while the identity table contains information about the user such as device type and browser. The transaction must exist in the transaction table, but might not always be available in the identity table.

The objective of the model is to predict if a transaction is fraudulent or not, given the transaction record.

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io

s3_client = boto3.client("s3", region_name=region)

# Importing existing dataset from AWS S3 locations
fraud_detection_bucket_name = f"sagemaker-example-files-prod-{region}"
identity_file_key = (
    "datasets/tabular/fraud_detection/synthethic_fraud_detection_SA/sampled_identity.csv"
)
transaction_file_key = (
    "datasets/tabular/fraud_detection/synthethic_fraud_detection_SA/sampled_transactions.csv"
)

identity_data_object = s3_client.get_object(
    Bucket=fraud_detection_bucket_name, Key=identity_file_key
)
transaction_data_object = s3_client.get_object(
    Bucket=fraud_detection_bucket_name, Key=transaction_file_key
)

identity_data = pd.read_csv(io.BytesIO(identity_data_object["Body"].read()))
transaction_data = pd.read_csv(io.BytesIO(transaction_data_object["Body"].read()))



In [9]:
pd.set_option('display.max_columns', None)

In [7]:
identity_data.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20
0,2990130,-5,38780.0,0.0,0.0,0.0,-70,0,1,100.0,36,32,80,253,241,260,125,T,F,F,T
1,2990266,-10,69246.0,0.0,0.0,0.0,-67,0,2,100.0,48,47,47,122,33,38,60,T,F,T,F
2,2992553,-45,348819.0,NaN,NaN,0.0,-73,0,0,100.0,47,21,143,268,111,2,135,F,F,T,F
3,2994568,-15,337170.0,NaN,NaN,0.0,-10,1,2,100.0,33,55,127,253,202,135,49,F,F,T,T
4,2994749,-5,680670.0,NaN,NaN,8.0,-1,2,2,100.0,29,52,43,257,7,19,254,F,F,T,T


In [8]:
transaction_data.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card4,card5,card6,B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11,B12,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,N1,N2,N3,N4,N5,N6,N7,N8,N9
0,3343087,0,8810855,29.00,12469,360.0,150.0,mastercard,126.0,debit,85,72,57,80,37,56,11,30,8,87,54,28,931,833,590,602,545,623,668,829,735,941,917,902,535,651,589,923,519,F,F,T,T,T,T,T,F,T
1,3307318,0,7955295,107.95,16188,178.0,150.0,mastercard,224.0,debit,78,2,52,36,67,86,35,62,45,8,0,46,786,786,587,726,956,649,921,555,965,804,971,711,880,858,822,654,773,F,T,T,T,F,F,F,F,T
2,3555327,0,15084339,159.95,1825,555.0,150.0,visa,226.0,debit,12,27,22,72,65,12,20,57,9,83,74,74,957,868,643,871,936,782,737,674,874,672,949,729,591,816,751,740,771,F,T,F,F,T,T,T,T,F
3,3310736,0,8017157,159.95,10057,225.0,150.0,mastercard,224.0,debit,56,67,41,63,80,58,23,100,71,79,7,13,831,610,947,973,895,538,685,771,801,929,818,758,593,662,907,550,903,T,T,F,T,T,F,T,F,F
4,3034711,0,1127470,117.00,11444,555.0,150.0,visa,226.0,debit,71,1,66,99,25,65,49,88,22,65,43,86,546,562,533,711,828,591,904,589,652,951,784,935,506,536,690,898,579,T,T,T,F,T,F,T,F,F


# Feature Engineering

In [11]:
# Feature Engineering

# Rounding off to 5 decimal places
identity_data = identity_data.round(5)
transaction_data = transaction_data.round(5)

# Filling null values with 0
identity_data = identity_data.fillna(0)
transaction_data = transaction_data.fillna(0)

# Feature transformations for this dataset are applied before ingestion into FeatureStore.
# One hot encode card4, card6
encoded_card_bank = pd.get_dummies(transaction_data["card4"], prefix="card_bank", dtype=np.uint8)
encoded_card_type = pd.get_dummies(transaction_data["card6"], prefix="card_type", dtype=np.uint8)

transformed_transaction_data = pd.concat(
    [transaction_data, encoded_card_type, encoded_card_bank], axis=1
)
# blank space is not allowed in feature name
transformed_transaction_data = transformed_transaction_data.rename(
    columns={"card_bank_american express": "card_bank_american_express"}
)

In [12]:
identity_data.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20
0,2990130,-5,38780.0,0.0,0.0,0.0,-70,0,1,100.0,36,32,80,253,241,260,125,T,F,F,T
1,2990266,-10,69246.0,0.0,0.0,0.0,-67,0,2,100.0,48,47,47,122,33,38,60,T,F,T,F
2,2992553,-45,348819.0,0.0,0.0,0.0,-73,0,0,100.0,47,21,143,268,111,2,135,F,F,T,F
3,2994568,-15,337170.0,0.0,0.0,0.0,-10,1,2,100.0,33,55,127,253,202,135,49,F,F,T,T
4,2994749,-5,680670.0,0.0,0.0,8.0,-1,2,2,100.0,29,52,43,257,7,19,254,F,F,T,T


In [13]:
transformed_transaction_data.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card4,card5,card6,B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11,B12,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,N1,N2,N3,N4,N5,N6,N7,N8,N9,card_type_0,card_type_credit,card_type_debit,card_bank_0,card_bank_american_express,card_bank_discover,card_bank_mastercard,card_bank_visa
0,3343087,0,8810855,29.00,12469,360.0,150.0,mastercard,126.0,debit,85,72,57,80,37,56,11,30,8,87,54,28,931,833,590,602,545,623,668,829,735,941,917,902,535,651,589,923,519,F,F,T,T,T,T,T,F,T,0,0,1,0,0,0,1,0
1,3307318,0,7955295,107.95,16188,178.0,150.0,mastercard,224.0,debit,78,2,52,36,67,86,35,62,45,8,0,46,786,786,587,726,956,649,921,555,965,804,971,711,880,858,822,654,773,F,T,T,T,F,F,F,F,T,0,0,1,0,0,0,1,0
2,3555327,0,15084339,159.95,1825,555.0,150.0,visa,226.0,debit,12,27,22,72,65,12,20,57,9,83,74,74,957,868,643,871,936,782,737,674,874,672,949,729,591,816,751,740,771,F,T,F,F,T,T,T,T,F,0,0,1,0,0,0,0,1
3,3310736,0,8017157,159.95,10057,225.0,150.0,mastercard,224.0,debit,56,67,41,63,80,58,23,100,71,79,7,13,831,610,947,973,895,538,685,771,801,929,818,758,593,662,907,550,903,T,T,F,T,T,F,T,F,F,0,0,1,0,0,0,1,0
4,3034711,0,1127470,117.00,11444,555.0,150.0,visa,226.0,debit,71,1,66,99,25,65,49,88,22,65,43,86,546,562,533,711,828,591,904,589,652,951,784,935,506,536,690,898,579,T,T,T,F,T,F,T,F,F,0,0,1,0,0,0,0,1


## Ingest Data into FeatureStore


In this step we will create the FeatureGroups representing the transaction and identity tables.

#### Define FeatureGroups

In [15]:
from time import gmtime, strftime, sleep

# creating unique feature group names
identity_feature_group_name = "identity-feature-group-" + strftime("%d-%H-%M-%S", gmtime())
transaction_feature_group_name = "transaction-feature-group-" + strftime("%d-%H-%M-%S", gmtime())

identity_feature_group_name, transaction_feature_group_name

('identity-feature-group-18-16-03-15', 'transaction-feature-group-18-16-03-15')

In [16]:
from sagemaker.feature_store.feature_group import FeatureGroup

# created feature groups in Feature Store
identity_feature_group = FeatureGroup(
    name=identity_feature_group_name, sagemaker_session=feature_store_session
)
transaction_feature_group = FeatureGroup(
    name=transaction_feature_group_name, sagemaker_session=feature_store_session
)

identity_feature_group, transaction_feature_group

(FeatureGroup(name='identity-feature-group-18-16-03-15', sagemaker_session=<sagemaker.session.Session object at 0x7fc4293feae0>, feature_definitions=[]),
 FeatureGroup(name='transaction-feature-group-18-16-03-15', sagemaker_session=<sagemaker.session.Session object at 0x7fc4293feae0>, feature_definitions=[]))

In [17]:
import time

current_time_sec = int(round(time.time()))


def cast_object_to_string(data_frame):
    for label in data_frame.columns:
        if data_frame.dtypes[label] == "object":
            data_frame[label] = data_frame[label].astype("str").astype("string")


# cast object dtype to string. The SageMaker FeatureStore Python SDK will then map the string dtype to String feature type.
cast_object_to_string(identity_data)
cast_object_to_string(transformed_transaction_data)

# record identifier and event time feature names
record_identifier_feature_name = "TransactionID"
event_time_feature_name = "EventTime"

# append EventTime feature
identity_data[event_time_feature_name] = pd.Series(
    [current_time_sec] * len(identity_data), dtype="float64"
)
transformed_transaction_data[event_time_feature_name] = pd.Series(
    [current_time_sec] * len(transaction_data), dtype="float64"
)

# load feature definitions to the feature group. SageMaker FeatureStore Python SDK will auto-detect the data schema based on input data.
identity_feature_group.load_feature_definitions(data_frame=identity_data)
# output is suppressed
transaction_feature_group.load_feature_definitions(data_frame=transformed_transaction_data)
# output is suppressed

[FeatureDefinition(feature_name='TransactionID', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='isFraud', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='TransactionDT', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='TransactionAmt', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='card1', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='card2', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='card3', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='card4', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_na

#### Create FeatureGroups in SageMaker FeatureStore

In [18]:
def wait_for_feature_group_creation_complete(feature_group):
    status = feature_group.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group Creation")
        time.sleep(5)
        status = feature_group.describe().get("FeatureGroupStatus")
    if status != "Created":
        raise RuntimeError(f"Failed to create feature group {feature_group.name}")
    print(f"FeatureGroup {feature_group.name} successfully created.")

# Creating Feature Group
identity_feature_group.create(
    s3_uri=f"s3://{default_s3_bucket_name}/{prefix}",
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name=event_time_feature_name,
    role_arn=role,
    enable_online_store=True,
)

transaction_feature_group.create(
    s3_uri=f"s3://{default_s3_bucket_name}/{prefix}",
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name=event_time_feature_name,
    role_arn=role,
    enable_online_store=True,
)

wait_for_feature_group_creation_complete(feature_group=identity_feature_group)
wait_for_feature_group_creation_complete(feature_group=transaction_feature_group)

Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
FeatureGroup identity-feature-group-18-16-03-15 successfully created.
Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
FeatureGroup transaction-feature-group-18-16-03-15 successfully created.


Confirm the FeatureGroup has been created by using the DescribeFeatureGroup and ListFeatureGroups APIs.

In [19]:
identity_feature_group.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:087941614028:feature-group/identity-feature-group-18-16-03-15',
 'FeatureGroupName': 'identity-feature-group-18-16-03-15',
 'RecordIdentifierFeatureName': 'TransactionID',
 'EventTimeFeatureName': 'EventTime',
 'FeatureDefinitions': [{'FeatureName': 'TransactionID',
   'FeatureType': 'Integral'},
  {'FeatureName': 'id_01', 'FeatureType': 'Integral'},
  {'FeatureName': 'id_02', 'FeatureType': 'Fractional'},
  {'FeatureName': 'id_03', 'FeatureType': 'Fractional'},
  {'FeatureName': 'id_04', 'FeatureType': 'Fractional'},
  {'FeatureName': 'id_05', 'FeatureType': 'Fractional'},
  {'FeatureName': 'id_06', 'FeatureType': 'Integral'},
  {'FeatureName': 'id_07', 'FeatureType': 'Integral'},
  {'FeatureName': 'id_08', 'FeatureType': 'Integral'},
  {'FeatureName': 'id_09', 'FeatureType': 'Fractional'},
  {'FeatureName': 'id_10', 'FeatureType': 'Integral'},
  {'FeatureName': 'id_11', 'FeatureType': 'Integral'},
  {'FeatureName': 'id_12', 'FeatureTyp

In [20]:
transaction_feature_group.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:087941614028:feature-group/transaction-feature-group-18-16-03-15',
 'FeatureGroupName': 'transaction-feature-group-18-16-03-15',
 'RecordIdentifierFeatureName': 'TransactionID',
 'EventTimeFeatureName': 'EventTime',
 'FeatureDefinitions': [{'FeatureName': 'TransactionID',
   'FeatureType': 'Integral'},
  {'FeatureName': 'isFraud', 'FeatureType': 'Integral'},
  {'FeatureName': 'TransactionDT', 'FeatureType': 'Integral'},
  {'FeatureName': 'TransactionAmt', 'FeatureType': 'Fractional'},
  {'FeatureName': 'card1', 'FeatureType': 'Integral'},
  {'FeatureName': 'card2', 'FeatureType': 'Fractional'},
  {'FeatureName': 'card3', 'FeatureType': 'Fractional'},
  {'FeatureName': 'card4', 'FeatureType': 'String'},
  {'FeatureName': 'card5', 'FeatureType': 'Fractional'},
  {'FeatureName': 'card6', 'FeatureType': 'String'},
  {'FeatureName': 'B1', 'FeatureType': 'Integral'},
  {'FeatureName': 'B2', 'FeatureType': 'Integral'},
  {'FeatureName': 'B3', '

In [21]:
sagemaker_client.list_feature_groups()  # use boto client to list FeatureGroups

{'FeatureGroupSummaries': [{'FeatureGroupName': 'transaction-feature-group-18-16-03-15',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:087941614028:feature-group/transaction-feature-group-18-16-03-15',
   'CreationTime': datetime.datetime(2026, 2, 18, 16, 9, 11, 149000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created'},
  {'FeatureGroupName': 'identity-feature-group-18-16-03-15',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:087941614028:feature-group/identity-feature-group-18-16-03-15',
   'CreationTime': datetime.datetime(2026, 2, 18, 16, 9, 9, 371000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created'}],
 'ResponseMetadata': {'RequestId': 'eccef498-4057-46bd-8cf6-0f29e4b29692',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'eccef498-4057-46bd-8cf6-0f29e4b29692',
   'strict-transport-security': 'max-age=47304000; includeSubDomains',
   'x-frame-options': 'DENY',
   'content-security-policy': "frame-ancestors 'none'",
   'cache-control': 'no-cache, n

#### PutRecords into FeatureGroup

After the FeatureGroups have been created, we can put data into the FeatureGroups by using the PutRecord API. This API can handle high TPS and is designed to be called by different streams. The data from all of these Put requests is buffered and written to S3 in chunks. The files will be written to the offline store within a few minutes of ingestion. For this example, to accelerate the ingestion process, we are specifying multiple workers to do the job simultaneously. It will take ~1min to ingest data to the 2 FeatureGroups, respectively.

In [22]:
identity_feature_group.ingest(data_frame=identity_data, max_workers=3, wait=True)

IngestionManagerPandas(feature_group_name='identity-feature-group-18-16-03-15', feature_definitions={'TransactionID': {'FeatureName': 'TransactionID', 'FeatureType': 'Integral'}, 'id_01': {'FeatureName': 'id_01', 'FeatureType': 'Integral'}, 'id_02': {'FeatureName': 'id_02', 'FeatureType': 'Fractional'}, 'id_03': {'FeatureName': 'id_03', 'FeatureType': 'Fractional'}, 'id_04': {'FeatureName': 'id_04', 'FeatureType': 'Fractional'}, 'id_05': {'FeatureName': 'id_05', 'FeatureType': 'Fractional'}, 'id_06': {'FeatureName': 'id_06', 'FeatureType': 'Integral'}, 'id_07': {'FeatureName': 'id_07', 'FeatureType': 'Integral'}, 'id_08': {'FeatureName': 'id_08', 'FeatureType': 'Integral'}, 'id_09': {'FeatureName': 'id_09', 'FeatureType': 'Fractional'}, 'id_10': {'FeatureName': 'id_10', 'FeatureType': 'Integral'}, 'id_11': {'FeatureName': 'id_11', 'FeatureType': 'Integral'}, 'id_12': {'FeatureName': 'id_12', 'FeatureType': 'Integral'}, 'id_13': {'FeatureName': 'id_13', 'FeatureType': 'Integral'}, 'id_1

In [23]:
transaction_feature_group.ingest(data_frame=transformed_transaction_data, max_workers=5, wait=True)

IngestionManagerPandas(feature_group_name='transaction-feature-group-18-16-03-15', feature_definitions={'TransactionID': {'FeatureName': 'TransactionID', 'FeatureType': 'Integral'}, 'isFraud': {'FeatureName': 'isFraud', 'FeatureType': 'Integral'}, 'TransactionDT': {'FeatureName': 'TransactionDT', 'FeatureType': 'Integral'}, 'TransactionAmt': {'FeatureName': 'TransactionAmt', 'FeatureType': 'Fractional'}, 'card1': {'FeatureName': 'card1', 'FeatureType': 'Integral'}, 'card2': {'FeatureName': 'card2', 'FeatureType': 'Fractional'}, 'card3': {'FeatureName': 'card3', 'FeatureType': 'Fractional'}, 'card4': {'FeatureName': 'card4', 'FeatureType': 'String'}, 'card5': {'FeatureName': 'card5', 'FeatureType': 'Fractional'}, 'card6': {'FeatureName': 'card6', 'FeatureType': 'String'}, 'B1': {'FeatureName': 'B1', 'FeatureType': 'Integral'}, 'B2': {'FeatureName': 'B2', 'FeatureType': 'Integral'}, 'B3': {'FeatureName': 'B3', 'FeatureType': 'Integral'}, 'B4': {'FeatureName': 'B4', 'FeatureType': 'Integr

To confirm that data has been ingested, we can quickly retrieve a record from the online store:

In [24]:
record_identifier_value = str(2990130)

featurestore_runtime.get_record(
    FeatureGroupName=transaction_feature_group_name,
    RecordIdentifierValueAsString=record_identifier_value,
)

{'ResponseMetadata': {'RequestId': 'e9ff0aa0-f23a-4293-acee-7bb375024db2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e9ff0aa0-f23a-4293-acee-7bb375024db2',
   'content-type': 'application/json',
   'content-length': '4078',
   'date': 'Wed, 18 Feb 2026 16:12:45 GMT'},
  'RetryAttempts': 0},
 'Record': [{'FeatureName': 'TransactionID', 'ValueAsString': '2990130'},
  {'FeatureName': 'isFraud', 'ValueAsString': '0'},
  {'FeatureName': 'TransactionDT', 'ValueAsString': '152647'},
  {'FeatureName': 'TransactionAmt', 'ValueAsString': '75.0'},
  {'FeatureName': 'card1', 'ValueAsString': '4577'},
  {'FeatureName': 'card2', 'ValueAsString': '583.0'},
  {'FeatureName': 'card3', 'ValueAsString': '150.0'},
  {'FeatureName': 'card4', 'ValueAsString': 'mastercard'},
  {'FeatureName': 'card5', 'ValueAsString': '219.0'},
  {'FeatureName': 'card6', 'ValueAsString': 'credit'},
  {'FeatureName': 'B1', 'ValueAsString': '69'},
  {'FeatureName': 'B2', 'ValueAsString': '80'},
  {'Featur

We can also retrieve a record of each feature group from the online store:

In [25]:
featurestore_runtime.batch_get_record(
    Identifiers=[
        {
            "FeatureGroupName": identity_feature_group_name,
            "RecordIdentifiersValueAsString": ["2990130"],
        },
        {
            "FeatureGroupName": transaction_feature_group_name,
            "RecordIdentifiersValueAsString": ["2990130"],
        },
    ]
)

{'ResponseMetadata': {'RequestId': '0a6105ac-a15e-4946-9cf7-ab015511ced2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0a6105ac-a15e-4946-9cf7-ab015511ced2',
   'content-type': 'application/json',
   'content-length': '5938',
   'date': 'Wed, 18 Feb 2026 16:13:30 GMT'},
  'RetryAttempts': 0},
 'Records': [{'FeatureGroupName': 'identity-feature-group-18-16-03-15',
   'RecordIdentifierValueAsString': '2990130',
   'Record': [{'FeatureName': 'TransactionID', 'ValueAsString': '2990130'},
    {'FeatureName': 'id_01', 'ValueAsString': '-5'},
    {'FeatureName': 'id_02', 'ValueAsString': '38780.0'},
    {'FeatureName': 'id_03', 'ValueAsString': '0.0'},
    {'FeatureName': 'id_04', 'ValueAsString': '0.0'},
    {'FeatureName': 'id_05', 'ValueAsString': '0.0'},
    {'FeatureName': 'id_06', 'ValueAsString': '-70'},
    {'FeatureName': 'id_07', 'ValueAsString': '0'},
    {'FeatureName': 'id_08', 'ValueAsString': '1'},
    {'FeatureName': 'id_09', 'ValueAsString': '100.0'},
    

The SageMaker Python SDK’s FeatureStore class also provides the functionality to generate Hive DDL commands. Schema of the table is generated based on the feature definitions. Columns are named after feature name and data-type are inferred based on feature type.

In [26]:
print(identity_feature_group.as_hive_ddl())

CREATE EXTERNAL TABLE IF NOT EXISTS sagemaker_featurestore.identity-feature-group-18-16-03-15 (
  TransactionID INT
  id_01 INT
  id_02 FLOAT
  id_03 FLOAT
  id_04 FLOAT
  id_05 FLOAT
  id_06 INT
  id_07 INT
  id_08 INT
  id_09 FLOAT
  id_10 INT
  id_11 INT
  id_12 INT
  id_13 INT
  id_14 INT
  id_15 INT
  id_16 INT
  id_17 STRING
  id_18 STRING
  id_19 STRING
  id_20 STRING
  EventTime FLOAT
  write_time TIMESTAMP
  event_time TIMESTAMP
  is_deleted BOOLEAN
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
  STORED AS
  INPUTFORMAT 'parquet.hive.DeprecatedParquetInputFormat'
  OUTPUTFORMAT 'parquet.hive.DeprecatedParquetOutputFormat'
LOCATION 's3://sagemaker-us-east-1-087941614028/sagemaker-featurestore-demo/087941614028/sagemaker/us-east-1/offline-store/identity-feature-group-18-16-03-15-1771430949/data'


In [27]:
print(transaction_feature_group.as_hive_ddl())

CREATE EXTERNAL TABLE IF NOT EXISTS sagemaker_featurestore.transaction-feature-group-18-16-03-15 (
  TransactionID INT
  isFraud INT
  TransactionDT INT
  TransactionAmt FLOAT
  card1 INT
  card2 FLOAT
  card3 FLOAT
  card4 STRING
  card5 FLOAT
  card6 STRING
  B1 INT
  B2 INT
  B3 INT
  B4 INT
  B5 INT
  B6 INT
  B7 INT
  B8 INT
  B9 INT
  B10 INT
  B11 INT
  B12 INT
  F1 INT
  F2 INT
  F3 INT
  F4 INT
  F5 INT
  F6 INT
  F7 INT
  F8 INT
  F9 INT
  F10 INT
  F11 INT
  F12 INT
  F13 INT
  F14 INT
  F15 INT
  F16 INT
  F17 INT
  N1 STRING
  N2 STRING
  N3 STRING
  N4 STRING
  N5 STRING
  N6 STRING
  N7 STRING
  N8 STRING
  N9 STRING
  card_type_0 INT
  card_type_credit INT
  card_type_debit INT
  card_bank_0 INT
  card_bank_american_express INT
  card_bank_discover INT
  card_bank_mastercard INT
  card_bank_visa INT
  EventTime FLOAT
  write_time TIMESTAMP
  event_time TIMESTAMP
  is_deleted BOOLEAN
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
  STORE

Now let's wait for the data to appear in our offline store before moving forward to creating a dataset. This will take approximately 5 minutes.

In [28]:
account_id = boto3.client("sts").get_caller_identity()["Account"]
print(account_id)

identity_feature_group_resolved_output_s3_uri = (
    identity_feature_group.describe()
    .get("OfflineStoreConfig")
    .get("S3StorageConfig")
    .get("ResolvedOutputS3Uri")
)
transaction_feature_group_resolved_output_s3_uri = (
    transaction_feature_group.describe()
    .get("OfflineStoreConfig")
    .get("S3StorageConfig")
    .get("ResolvedOutputS3Uri")
)

identity_feature_group_s3_prefix = identity_feature_group_resolved_output_s3_uri.replace(
    f"s3://{default_s3_bucket_name}/", ""
)
transaction_feature_group_s3_prefix = transaction_feature_group_resolved_output_s3_uri.replace(
    f"s3://{default_s3_bucket_name}/", ""
)

offline_store_contents = None
while offline_store_contents is None:
    objects_in_bucket = s3_client.list_objects(
        Bucket=default_s3_bucket_name, Prefix=transaction_feature_group_s3_prefix
    )
    if "Contents" in objects_in_bucket and len(objects_in_bucket["Contents"]) > 1:
        offline_store_contents = objects_in_bucket["Contents"]
    else:
        print("Waiting for data in offline store...\n")
        sleep(60)

print("Data available.")

087941614028
Waiting for data in offline store...

Waiting for data in offline store...

Data available.


SageMaker FeatureStore adds metadata for each record that's ingested into the offline store.

## Build Training Dataset

SageMaker FeatureStore automatically builds the Glue Data Catalog for FeatureGroups (you can optionally turn it on/off while creating the FeatureGroup). In this example, we want to create one training dataset with FeatureValues from both identity and transaction FeatureGroups. This is done by utilizing the auto-built Catalog. We run an Athena query that joins the data stored in the offline store in S3 from the 2 FeatureGroups. 

In [29]:
identity_query = identity_feature_group.athena_query()
transaction_query = transaction_feature_group.athena_query()

identity_table = identity_query.table_name
transaction_table = transaction_query.table_name

query_string = (
    'SELECT * FROM "'
    + transaction_table
    + '" LEFT JOIN "'
    + identity_table
    + '" ON "'
    + transaction_table
    + '".transactionid = "'
    + identity_table
    + '".transactionid'
)
print("Running " + query_string)

# run Athena query. The output is loaded to a Pandas dataframe.
# dataset = pd.DataFrame()
identity_query.run(
    query_string=query_string,
    output_location="s3://" + default_s3_bucket_name + "/" + prefix + "/query_results/",
)
identity_query.wait()
dataset = identity_query.as_dataframe()

dataset

Running SELECT * FROM "transaction_feature_group_18_16_03_15_1771430951" LEFT JOIN "identity_feature_group_18_16_03_15_1771430949" ON "transaction_feature_group_18_16_03_15_1771430951".transactionid = "identity_feature_group_18_16_03_15_1771430949".transactionid


,transactionid,isfraud,transactiondt,transactionamt,card1,card2,card3,card4,card5,card6,b1,b2,b3,b4,b5,b6,b7,b8,b9,b10,b11,b12,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,n1,n2,n3,n4,n5,n6,n7,n8,n9,card_type_0,card_type_credit,card_type_debit,card_bank_0,card_bank_american_express,card_bank_discover,card_bank_mastercard,card_bank_visa,eventtime,write_time,api_invocation_time,is_deleted,transactionid.1,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,eventtime.1,write_time.1,api_invocation_time.1,is_deleted.1
0,3010634,0,602155,35.00,2544,322.0,150.0,visa,226.0,credit,39,5,91,9,59,40,68,36,56,38,61,37,796,943,707,691,583,597,845,711,701,740,816,770,816,620,605,803,743,T,F,F,F,T,T,T,F,T,0,1,0,0,0,0,0,1,1.771431e+09,2026-02-18 16:17:08.575,2026-02-18 16:12:17.000,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3457873,0,12158334,35.95,10682,514.0,150.0,mastercard,224.0,debit,17,48,72,92,7,36,43,94,33,26,72,95,794,590,841,791,701,967,932,873,837,994,669,841,568,510,794,649,581,F,F,F,T,T,F,F,F,T,0,0,1,0,0,0,1,0,1.771431e+09,2026-02-18 16:17:08.477,2026-02-18 16:12:18.000,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3378135,0,9823621,92.00,7919,194.0,150.0,mastercard,166.0,debit,84,73,50,80,99,43,73,29,84,19,42,67,724,903,954,897,862,860,639,642,587,723,755,687,967,731,637,907,546,T,T,T,F,T,T,T,F,F,0,0,1,0,0,0,1,0,1.771431e+09,2026-02-18 16:17:18.518,2026-02-18 16:12:18.000,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3426764,0,11140157,100.00,5293,321.0,150.0,visa,226.0,debit,93,16,59,77,28,7,69,38,66,38,1,67,954,948,823,608,884,980,793,893,720,585,828,575,744,811,872,814,875,F,F,F,T,F,T,F,T,T,0,0,1,0,0,0,0,1,1.771431e+09,2026-02-18 16:17:08.512,2026-02-18 16:12:17.000,False,3426764.0,0.0,98933.0,0.0,0.0,0.0,-7.0,0.0,0.0,100.0,35.0,53.0,65.0,129.0,83.0,286.0,159.0,T,F,T,T,1.771431e+09,2026-02-18 16:17:11.571,2026-02-18 16:12:15.000,False
4,3140270,0,3177093,319.95,15497,490.0,150.0,visa,226.0,debit,96,54,60,0,70,78,67,55,18,73,99,71,546,965,507,816,526,917,582,911,529,856,897,931,581,611,629,740,649,F,F,F,T,F,F,T,T,T,0,0,1,0,0,0,0,1,1.771431e+09,2026-02-18 16:17:18.518,2026-02-18 16:12:18.000,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,3500743,0,13453239,48.95,17838,555.0,150.0,visa,226.0,debit,66,88,87,46,30,41,0,89,64,41,87,62,782,612,606,810,768,728,816,941,860,909,713,585,999,543,768,980,772,F,F,T,F,F,T,F,T,F,0,0,1,0,0,0,0,1,1.771431e+09,2026-02-18 16:17:08.478,2026-02-18 16:12:21.000,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,3219359,0,5508738,134.95,17217,111.0,150.0,visa,226.0,debit,12,32,90,43,89,28,29,34,88,79,21,100,645,973,557,948,556,851,900,878,559,605,641,535,580,549,838,844,946,T,T,F,T,F,F,T,T,F,0,0,1,0,0,0,0,1,1.771431e+09,2026-02-18 16:17:08.478,2026-02-18 16:12:21.000,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,3296020,1,7685133,884.00,13926,327.0,150.0,discover,142.0,credit,27,43,73,26,65,13,34,94,41,6,39,1,636,768,984,594,502,821,793,501,890,771,778,684,853,861,909,748,725,F,T,T,F,F,F,F,F,T,0,1,0,0,0,1,0,0,1.771431e+09,2026-02-18 16:17:08.478,2026-02-18 16:12:22.000,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998,3419489,0,10941516,34.00,7508,321.0

In [30]:
# Prepare query results for training.
query_execution = identity_query.get_query_execution()
query_result = (
    "s3://"
    + default_s3_bucket_name
    + "/"
    + prefix
    + "/query_results/"
    + query_execution["QueryExecution"]["QueryExecutionId"]
    + ".csv"
)
print(query_result)

# Select useful columns for training with target column as the first.
dataset = dataset[
    [
        "isfraud",
        "transactiondt",
        "transactionamt",
        "card1",
        "card2",
        "card3",
        "card5",
        "card_type_credit",
        "card_type_debit",
        "card_bank_american_express",
        "card_bank_discover",
        "card_bank_mastercard",
        "card_bank_visa",
        "id_01",
        "id_02",
        "id_03",
        "id_04",
        "id_05",
    ]
]

# Write to csv in S3 without headers and index column.
dataset.to_csv("dataset.csv", header=False, index=False)
s3_client.upload_file("dataset.csv", default_s3_bucket_name, prefix + "/training_input/dataset.csv")
dataset_uri_prefix = "s3://" + default_s3_bucket_name + "/" + prefix + "/training_input/"

dataset

s3://sagemaker-us-east-1-087941614028/sagemaker-featurestore-demo/query_results/f2fe42f2-0608-4b45-893a-b1abfcb80a0d.csv


,isfraud,transactiondt,transactionamt,card1,card2,card3,card5,card_type_credit,card_type_debit,card_bank_american_express,card_bank_discover,card_bank_mastercard,card_bank_visa,id_01,id_02,id_03,id_04,id_05
0,0,602155,35.00,2544,322.0,150.0,226.0,1,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN
1,0,12158334,35.95,10682,514.0,150.0,224.0,0,1,0,0,1,0,NaN,NaN,NaN,NaN,NaN
2,0,9823621,92.00,7919,194.0,150.0,166.0,0,1,0,0,1,0,NaN,NaN,NaN,NaN,NaN
3,0,11140157,100.00,5293,321.0,150.0,226.0,0,1,0,0,0,1,0.0,98933.0,0.0,0.0,0.0
4,0,3177093,319.95,15497,490.0,150.0,226.0,0,1,0,0,0,1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0,13453239,48.95,17838,555.0,150.0,226.0,0,1,0,0,0,1,NaN,NaN,NaN,NaN,NaN
1996,0,5508738,134.95,17217,111.0,150.0,226.0,0,1,0,0,0,1,NaN,NaN,NaN,NaN,NaN
1997,1,7685133,884.00,13926,327.0,150.0,142.0,1,0,0,1,0,0,NaN,NaN,NaN,NaN,NaN
1998,0,10941516,34.00,7508,321.0,150.0,226.0,0,1,0,0,0,1,NaN,NaN,NaN,NaN,NaN


## Train and Deploy the Model

Now it's time to launch a Training job to fit our model. We use the gradient boosting algorithm provided by XGBoost libary to fit our data. Call the SageMaker XGBoost container and construct a generic SageMaker estimator.

In [31]:
training_image = sagemaker.image_uris.retrieve("xgboost", region, "1.0-1")

#### Construct a SageMaker generic estimator using the SageMaker XGBoost container

In [32]:
training_output_path = "s3://" + default_s3_bucket_name + "/" + prefix + "/training_output"

from sagemaker.estimator import Estimator

training_model = Estimator(
    training_image,
    role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size=5,
    max_run=3600,
    input_mode="File",
    output_path=training_output_path,
    sagemaker_session=feature_store_session,
)

#### Set hyperparameters

In [33]:
training_model.set_hyperparameters(objective="binary:logistic", num_round=50)

#### Specify training dataset 
Specify the training dataset created in the [Build Training Dataset](#Build-Training-Dataset) section.

In [34]:
train_data = sagemaker.inputs.TrainingInput(
    dataset_uri_prefix,
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
)
data_channels = {"train": train_data}

#### Start training

In [35]:
training_model.fit(inputs=data_channels, logs=True)

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2026-02-18-16-26-35-494


2026-02-18 16:26:36 Starting - Starting the training job...
2026-02-18 16:26:51 Starting - Preparing the instances for training...
2026-02-18 16:27:40 Downloading - Downloading the training image......
2026-02-18 16:28:36 Training - Training image download completed. Training in progress.
2026-02-18 16:28:36 Uploading - Uploading generated training model.[2026-02-18 16:28:31.913 ip-10-0-77-139.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Single node training.
[16:28:31] 2000x17 matrix with 324

## Set up Hosting for the Model

Once the training is done, we can deploy the trained model as an Amazon SageMaker real-time hosted endpoint. This will allow us to make predictions (or inference) from the model. Note that we don't have to host on the same instance (or type of instance) that we used to train. The endpoint deployment can be accomplished as follows. This takes 8-10 minutes to complete.

In [36]:
predictor = training_model.deploy(initial_instance_count=1, instance_type="ml.m5.xlarge")

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2026-02-18-16-38-40-636
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2026-02-18-16-38-40-636
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2026-02-18-16-38-40-636


------!

## SageMaker FeatureStore During Inference

SageMaker FeatureStore can be useful in supplementing data for inference requests because of the low-latency GetRecord functionality. For this demo, we will be given a TransactionId and query our online FeatureGroups for data on the transaction to build our inference request. 


In [37]:
# Incoming inference request.
transaction_id = str(3450774)


# Helper to parse the feature value from the record.
def get_feature_value(record, feature_name):
    return str(list(filter(lambda r: r["FeatureName"] == feature_name, record))[0]["ValueAsString"])


transaction_response = featurestore_runtime.get_record(
    FeatureGroupName=transaction_feature_group_name, RecordIdentifierValueAsString=transaction_id
)
transaction_record = transaction_response["Record"]

transaction_test_data = [
    get_feature_value(transaction_record, "TransactionDT"),
    get_feature_value(transaction_record, "TransactionAmt"),
    get_feature_value(transaction_record, "card1"),
    get_feature_value(transaction_record, "card2"),
    get_feature_value(transaction_record, "card3"),
    get_feature_value(transaction_record, "card5"),
    get_feature_value(transaction_record, "card_type_credit"),
    get_feature_value(transaction_record, "card_type_debit"),
    get_feature_value(transaction_record, "card_bank_american_express"),
    get_feature_value(transaction_record, "card_bank_discover"),
    get_feature_value(transaction_record, "card_bank_mastercard"),
    get_feature_value(transaction_record, "card_bank_visa"),
]

identity_response = featurestore_runtime.get_record(
    FeatureGroupName=identity_feature_group_name, RecordIdentifierValueAsString=transaction_id
)
identity_record = identity_response["Record"]
id_test_data = [
    get_feature_value(identity_record, "id_01"),
    get_feature_value(identity_record, "id_02"),
    get_feature_value(identity_record, "id_03"),
    get_feature_value(identity_record, "id_04"),
    get_feature_value(identity_record, "id_05"),
]

# Join all pieces for inference request.
inference_request = []
inference_request.extend(transaction_test_data[:])
inference_request.extend(id_test_data[:])

inference_request

['11923451',
 '50.0',
 '12501',
 '490.0',
 '150.0',
 '226.0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '-40',
 '20130.0',
 '0.0',
 '0.0',
 '16.0']

In [38]:
import json

results = predictor.predict(",".join(inference_request), initial_args={"ContentType": "text/csv"})
prediction = json.loads(results)
print(prediction)

0.8026058077812195


## Cleanup Resources

In [39]:
predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: sagemaker-xgboost-2026-02-18-16-38-40-636
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2026-02-18-16-38-40-636


In [40]:
identity_feature_group.delete()
transaction_feature_group.delete()

In [41]:
# restore original boto3 version
%pip install 'boto3=={}'.format(original_boto3_version)

/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `/opt/conda/bin/python -m pip install 'boto3=={}'.format(original_boto3_version)'
Note: you may need to restart the kernel to use updated packages.


## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-1/sagemaker-featurestore|sagemaker_featurestore_fraud_detection_python_sdk.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-2/sagemaker-featurestore|sagemaker_featurestore_fraud_detection_python_sdk.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-1/sagemaker-featurestore|sagemaker_featurestore_fraud_detection_python_sdk.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ca-central-1/sagemaker-featurestore|sagemaker_featurestore_fraud_detection_python_sdk.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/sa-east-1/sagemaker-featurestore|sagemaker_featurestore_fraud_detection_python_sdk.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-1/sagemaker-featurestore|sagemaker_featurestore_fraud_detection_python_sdk.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-2/sagemaker-featurestore|sagemaker_featurestore_fraud_detection_python_sdk.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-3/sagemaker-featurestore|sagemaker_featurestore_fraud_detection_python_sdk.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-central-1/sagemaker-featurestore|sagemaker_featurestore_fraud_detection_python_sdk.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-north-1/sagemaker-featurestore|sagemaker_featurestore_fraud_detection_python_sdk.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-1/sagemaker-featurestore|sagemaker_featurestore_fraud_detection_python_sdk.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-2/sagemaker-featurestore|sagemaker_featurestore_fraud_detection_python_sdk.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-1/sagemaker-featurestore|sagemaker_featurestore_fraud_detection_python_sdk.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-2/sagemaker-featurestore|sagemaker_featurestore_fraud_detection_python_sdk.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-south-1/sagemaker-featurestore|sagemaker_featurestore_fraud_detection_python_sdk.ipynb)
